In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf model

GIT_PATH = "https://github.com/rahulsinghal1904/Transformation-of-Natural-Language-Questions-to-SQL-Queries/"
!git clone "{GIT_PATH}"


Cloning into 'Major-Project'...
remote: Enumerating objects: 30, done.
remote: Total 30 (delta 0), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (30/30), done.


In [ ]:
%cd Transformation-of-Natural-Language-Questions-to-SQL-Queries
%cd model

/content/Major-Project
/content/Major-Project/model


Downloading the dependencies

In [ ]:
!pip install SQLAlchemy==1.3.20
!pip install records==0.5.3
!pip install transformers==3.4.0

     |████████████████████████████████| 1.3 MB 4.0 MB/s 
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.4.27
    Uninstalling SQLAlchemy-1.4.27:
      Successfully uninstalled SQLAlchemy-1.4.27
     |████████████████████████████████| 158 kB 5.9 MB/s 
     |████████████████████████████████| 48 kB 5.5 MB/s 
  Created wheel for openpyxl: filename=openpyxl-2.4.11-py2.py3-none-any.whl size=222839 sha256=ae7290c4bc2aa3308ee19f041e1d330b0b9acf22dad04a9fb1a463667f2bcb89
  Stored in directory: /root/.cache/pip/wheels/ee/c3/67/b0c36cbd1242ea70054d400cad5192f02728eb67014192fd85
Successfully built openpyxl
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 1.3 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
     |████████████████████████████████| 1.2 MB 37.6 MB/s 
     |█████████████████████████

Let us import the libraries that we will be using

In [ ]:
import load_data
import torch
import json,argparse
import load_model
import roberta_training
import corenlp_local
import seq2sql_model_testing
import seq2sql_model_training_functions
import model_save_and_infer
import dev_function
import infer_functions
import time
import os
import nltk

from dbengine_sqlnet import DBEngine
from torchsummary import summary
from tqdm.notebook import tqdm
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The following cell will set the PyTorch device to a GPU which enables us to use it during runtime.

In [ ]:
device = torch.device("cuda")
device

device(type='cuda')

## Loading Data From Files

In [ ]:
path_wikisql = "/content/drive/My Drive/Major Project Resources"
BATCH_SIZE = 8

train_data, train_table, dev_data, dev_table, train_loader, dev_loader = load_data.get_data(path_wikisql, batch_size = BATCH_SIZE)
test_data,test_table,test_loader = load_data.get_test_data(path_wikisql, batch_size = BATCH_SIZE)
zero_data,zero_table,zero_loader = load_data.get_zero_data(path_wikisql, batch_size = BATCH_SIZE)    # Data to test Zero Shot Learning

## Loading Models

In [ ]:
roberta_model, tokenizer, configuration = load_model.get_roberta_model()          # Loads the Optimized BERT Model
seq2sql_model = load_model.get_seq2sql_model(configuration.hidden_size)           # Loads the LSTM based submodels

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

## Loading the Pre trained weights

In [ ]:
path_roberta_pretrained = path_wikisql + "/model_roberta_best.pt"
path_model_pretrained = path_wikisql + "/model_best.pt"

if torch.cuda.is_available():
    res = torch.load(path_roberta_pretrained)
else:
    res = torch.load(path_roberta_pretrained, map_location='cpu')

roberta_model.load_state_dict(res['model_roberta'])

if torch.cuda.is_available():
    res = torch.load(path_model_pretrained)
else:
    res = torch.load(path_model_pretrained, map_location='cpu')

seq2sql_model.load_state_dict(res['model'])

<All keys matched successfully>

## Loading the Model Optimizers

##### RoBERTa: Adam Optimizer with learning rate = 0.00001
##### SubModels: Adam Optimizer with learning rate = 0.001

In [ ]:
model_optimizer, roberta_optimizer = load_model.get_optimizers(seq2sql_model , roberta_model)

## Below we define a function that prints the metrics 

In [ ]:
def print_result(epoch, acc, dname):
    ave_loss, acc_sc, acc_sa, acc_wn, acc_wc, acc_wo, acc_wvi, acc_wv, acc_lx, acc_x = acc

    print(f'{dname} results ------------')
    print(
        f" Epoch: {epoch}, ave loss: {ave_loss}, acc_sc: {acc_sc:.3f}, acc_sa: {acc_sa:.3f}, acc_wn: {acc_wn:.3f}, \
        acc_wc: {acc_wc:.3f}, acc_wo: {acc_wo:.3f}, acc_wvi: {acc_wvi:.3f}, acc_wv: {acc_wv:.3f}, acc_lx: {acc_lx:.3f}, acc_x: {acc_x:.3f}"
    )

## Training the Model

In [ ]:
EPOCHS = 1

In [ ]:
acc_lx_t_best = 0.7             # Creats checkpoint so that a worse model does not get saved
epoch_best = 0                   
for epoch in range(EPOCHS):
    acc_train = dev_function.train( seq2sql_model, roberta_model, model_optimizer, roberta_optimizer, tokenizer, configuration, path_wikisql, train_loader)
    acc_dev, results_dev, cnt_list = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, dev_loader, mode="dev")
    print_result(epoch, acc_train, 'train')
    print_result(epoch, acc_dev, 'dev')
    acc_lx_t = acc_dev[-2]
    if acc_lx_t > acc_lx_t_best:                  
        acc_lx_t_best = acc_lx_t                  
        epoch_best = epoch                        
          
        # save best model
        state = {'model': seq2sql_model.state_dict()}
        torch.save(state, os.path.join(path_wikisql, 'model_best.pt'))

        state = {'model_roberta': roberta_model.state_dict()}
        torch.save(state, os.path.join(path_wikisql, 'model_roberta_best.pt'))

    print(f" Best Dev lx acc: {acc_lx_t_best} at epoch: {epoch_best}")

  0%|          | 0/7045 [00:00<?, ?it/s]

  0%|          | 0/1053 [00:00<?, ?it/s]

train results ------------
 Epoch: 0, ave loss: 0.9225952073155633, acc_sc: 0.932, acc_sa: 0.908, acc_wn: 0.978,         acc_wc: 0.886, acc_wo: 0.883, acc_wvi: 0.695, acc_wv: 0.696, acc_lx: 0.546, acc_x: 0.632
dev results ------------
 Epoch: 0, ave loss: None, acc_sc: 0.956, acc_sa: 0.906, acc_wn: 0.978,         acc_wc: 0.843, acc_wo: 0.823, acc_wvi: 0.000, acc_wv: 0.673, acc_lx: 0.543, acc_x: 0.637
 Best Dev lx acc: 0.7 at epoch: 0


## Testing The Model

In [ ]:
acc_dev, results_dev, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, dev_loader, mode="dev")
acc_test, results_test, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, test_loader, mode="test")
acc_zero, results_zero, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, zero_loader, mode="test")

print_result('test', acc_dev, 'dev')
print_result('test', acc_test, 'test')
print_result('test', acc_zero, 'zero')

  0%|          | 0/1053 [00:00<?, ?it/s]

  0%|          | 0/1985 [00:00<?, ?it/s]

  0%|          | 0/900 [00:00<?, ?it/s]

dev results ------------
 Epoch: test, ave loss: None, acc_sc: 0.956, acc_sa: 0.906, acc_wn: 0.978,         acc_wc: 0.843, acc_wo: 0.823, acc_wvi: 0.000, acc_wv: 0.673, acc_lx: 0.543, acc_x: 0.637
test results ------------
 Epoch: test, ave loss: None, acc_sc: 0.950, acc_sa: 0.905, acc_wn: 0.972,         acc_wc: 0.836, acc_wo: 0.826, acc_wvi: 0.000, acc_wv: 0.672, acc_lx: 0.533, acc_x: 0.632
zero results ------------
 Epoch: test, ave loss: None, acc_sc: 0.950, acc_sa: 0.899, acc_wn: 0.971,         acc_wc: 0.821, acc_wo: 0.809, acc_wvi: 0.000, acc_wv: 0.646, acc_lx: 0.501, acc_x: 0.613


## Testing an example query

In [ ]:
nlu = "Which year did the band release the Song 'Wake me Up'?"

# Specify the Table Schema
table_id = '1-10015132-16'
headers = ['Band', 'Song', 'Studio', 'Year', 'Awards']
types = ['text', 'text', 'text', 'text', 'text']

In [ ]:
pr_sql_i =  infer_functions.infer(
                nlu,
                table_id, headers, types, tokenizer, 
                seq2sql_model, roberta_model, configuration, max_seq_length=222,
                num_target_layers=2,
                beam_size=4
            )

START ============================================================= 
[['Band', 'Song', 'Studio', 'Year', 'Awards']]
nlu: ["Which year did the band release the Song 'Wake me Up'?"]
pr_sql_i : [{'agg': 0, 'sel': 3, 'conds': [[0, 0, "'wake me up"]]}]
pr_sql_q : [["SELECT (Year) FROM 1-10015132-16 WHERE Band = 'wake me up"]]
---------------------------------------------------------------------


In [ ]:
nlu = "What was the slogan of the State 'Queensland'?"

# Specify the Table Schema
table_id = '1-1000181-1'
headers = ['State/territory', 'Text/background colour','Format','Current slogan','Current series','Notes']
types = ['text', 'text', 'text', 'text', 'text']

In [ ]:
pr_sql_i =  infer_functions.infer(
                nlu,
                table_id, headers, types, tokenizer, 
                seq2sql_model, roberta_model, configuration, max_seq_length=222,
                num_target_layers=2,
                beam_size=4
            )

START ============================================================= 
[['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']]
nlu: ["What was the slogan of the State 'Queensland'?"]
pr_sql_i : [{'agg': 0, 'sel': 3, 'conds': [[0, 0, "'queensland"]]}]
pr_sql_q : [["SELECT (Current slogan) FROM 1-1000181-1 WHERE State/territory = 'queensland"]]
---------------------------------------------------------------------


In [ ]:
!rm -rf model